In [39]:
%pip install scikit-learn

4078.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [40]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [41]:
# Load the dataset
df = pd.read_csv("datasets/diabetes.csv")

In [42]:
# Check for nulls and print shape
print("Initial shape:", df.shape)
print("Missing values per column:\n", df.isnull().sum())


Initial shape: (768, 9)
Missing values per column:
 Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [43]:
cols_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)
df.fillna(df.median(), inplace=True)

In [44]:
# Separate features and target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [45]:
# Normalize numerical features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)


In [46]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

In [47]:
#Save processed datasets (optional)
X_train.to_csv("datasets/X_train_diabetes.csv", index=False)
X_test.to_csv("datasets/X_test_diabetes.csv", index=False)
y_train.to_csv("datasets/y_train_diabetes.csv", index=False)
y_test.to_csv("datasets/y_test_diabetes.csv", index=False)

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train Random Forest
rf = RandomForestClassifier(
    n_estimators=100,      # Number of trees
    max_depth=5,           # Try tuning this
    random_state=42
)
rf.fit(X_train, y_train)

# Predict
y_pred_rf = rf.predict(X_test)

# Evaluate
print("✅ Accuracy:", accuracy_score(y_test, y_pred_rf))
print("📊 Classification Report:\n", classification_report(y_test, y_pred_rf))


✅ Accuracy: 0.7532467532467533
📊 Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.83      0.81        99
           1       0.67      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.73       154
weighted avg       0.75      0.75      0.75       154



In [49]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 10],
    'max_features': ['sqrt', 'log2']
}

grid_search_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=2
)

grid_search_rf.fit(X_train, y_train)

print("Best Parameters:", grid_search_rf.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.2s
[CV] END ...max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.2s
[CV] END ...max_depth=3, max_features=sqrt, n_es

In [50]:
# Train and evaluate Logistic Regression on diabetes data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train model
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Predict
y_pred_logreg = logreg.predict(X_test)

# Evaluate
print('Accuracy:', accuracy_score(y_test, y_pred_logreg))
print('Classification Report:')
print(classification_report(y_test, y_pred_logreg))

Accuracy: 0.7532467532467533
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.83      0.81        99
           1       0.67      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.73       154
weighted avg       0.75      0.75      0.75       154



In [51]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [52]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, max_depth=5,  random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)


In [53]:
ensemble_model = VotingClassifier(
    estimators=[
        ('lr', log_reg),
        ('rf', rf),
        ('xgb', xgb)
    ],
    voting='hard'  # or 'hard'
)

In [54]:
ensemble_model.fit(X_train, y_train)
y_pred = ensemble_model.predict(X_test)
print('Ensemble Model Accuracy:', accuracy_score(y_test, y_pred))
print('Ensemble Model Classification Report:')
print(classification_report(y_test, y_pred))

Ensemble Model Accuracy: 0.7532467532467533
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.82      0.81        99
           1       0.66      0.64      0.65        55

    accuracy                           0.75       154
   macro avg       0.73      0.73      0.73       154
weighted avg       0.75      0.75      0.75       154



/Users/anusreemohanan/.pyenv/versions/3.11.7/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:33:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [55]:
import pickle

# Save the model and scaler as a tuple
with open("diabetes_risk_model.pkl", "wb") as f:
    pickle.dump((scaler, ensemble_model), f)

print("Model and scaler saved successfully.")

Model and scaler saved successfully.
